# Discover the Higgs with Deep Neural Networks
# Chapter 3: Evaluate and Apply a Neural Network

In this chapter you will evaluate and apply your first neural network.

In [ ]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import seed
import os

# Import the tensorflow module to create a neural network
import tensorflow as tf
from tensorflow.data import Dataset

# Import some common functions created for this notebook
import common

# Random state
random_state = 42
_ = np.random.RandomState(random_state)

## Data Preparation

### Load the Data

In [ ]:
# Define the input samples
sample_list_signal = ['ggH125_ZZ4lep', 'VBFH125_ZZ4lep', 'WH125_ZZ4lep', 'ZH125_ZZ4lep']
sample_list_background = ['llll', 'Zee', 'Zmumu', 'ttbar_lep']

In [ ]:
sample_path = 'input'
# Read all the samples
no_selection_data_frames = {}
for sample in sample_list_signal + sample_list_background:
    no_selection_data_frames[sample] = pd.read_csv(os.path.join(sample_path, sample + '.csv'))

### Event Pre-Selection

Import the pre-selection functions saved during the first chapter. If the modules are not found solve and execute the notebook of the first chapter.

In [ ]:
from functions.selection_lepton_charge import selection_lepton_charge
from functions.selection_lepton_type import selection_lepton_type

In [ ]:
# Create a copy of the original data frame to investigate later
data_frames = no_selection_data_frames.copy()

# Apply the chosen selection criteria
for sample in sample_list_signal + sample_list_background:
    # Selection on lepton type
    type_selection = np.vectorize(selection_lepton_type)(
        data_frames[sample].lep1_pdgId,
        data_frames[sample].lep2_pdgId,
        data_frames[sample].lep3_pdgId,
        data_frames[sample].lep4_pdgId)
    data_frames[sample] = data_frames[sample][type_selection]

    # Selection on lepton charge
    charge_selection = np.vectorize(selection_lepton_charge)(
        data_frames[sample].lep1_charge,
        data_frames[sample].lep2_charge,
        data_frames[sample].lep3_charge,
        data_frames[sample].lep4_charge)
    data_frames[sample] = data_frames[sample][charge_selection]

### Get Test and Training Data

To avoid a bias we are going to keep 40% of our data for the final application. This test data should not be used to train or evaluate your model.

In [ ]:
# Split data to keep 40% for testing
train_data_frames, test_data_frames = common.split_data_frames(data_frames, 0.6)

## Load Neural Network

Now lets load the model trained in the chapter 2

In [ ]:
model = tf.keras.models.load_model('models/chapter2_model')

As you can see in the summary the architecture of the model is exactly the same as saved in the chapter before.

In [ ]:
# Display the model's architecture
model.summary()

## Application on Train Data

The model is trained, so the time has come to use it. First we extract the training data.

In [ ]:
# The training input variables
training_variables = ['lep1_pt', 'lep2_pt', 'lep3_pt', 'lep4_pt']

In [ ]:
# Extract the values and classification
values, _, classification = common.get_dnn_input(train_data_frames, training_variables, sample_list_signal, sample_list_background)

Let's compare the true classification of some random events and the prediction the model gives for these events.

In [ ]:
# Choose some random events
random_idx = [1841, 11852, 15297, 263217, 278357, 331697]
print('Classification:')
print(classification[random_idx])
print('Prediction')
print(model.predict(values[random_idx]))

<font color='blue'>
Task:

Get the prediction for all events and use <code>common.plot_dnn_output(...)</code> to plot the result.
</font>

In [ ]:
# Apply the model for all values
prediction = model.predict(values)

In [ ]:
# Plot the model output
common.plot_dnn_output(prediction, classification)
_ = plt.show()

## Application on Test Data

Use `common.apply_dnn_model(...)` to apply the model for all samples in `test_data_frames` and add the classification to the data frame. Afterwards the prediction can be plotted just as the other kinematic variables.

In [ ]:
# Apply the model
data_frames_apply_dnn = common.apply_dnn_model(model, test_data_frames, training_variables, sample_list_signal + sample_list_background)

After we have add the classification to the data frames we can plot the classification like any other observable.

In [ ]:
model_prediction = {'variable': 'model_prediction',
                    'binning': np.linspace(0, 1, 50),
                    'xlabel': 'prediction'}
common.plot_hist(model_prediction, data_frames_apply_dnn, show_data=False)
plt.show()

This is the final classification the neural network would give on unseen data. As you can see we are far from perfect. In the next chapters we will try to improve this classification